#랜드마크 가져오기..

- 뭘 바꿔야할지 모르겟네..

In [ ]:
import cv2
import dlib

def getLandmarks(im):
    #크기조정 (나중에 해보면서 다시 조정해야할 것 같습니다)
    imSmall = cv2.resize(im, None, 
                            fx = 1.0, 
                            fy = 1.0, 
                            interpolation = cv2.INTER_LINEAR)   #쌍선형 보간법

    #얼굴 인식을 먼저하고.. rects에 사각형의 꼭짓점 좌표가 저장됨..
    rects = detector(imSmall, 0)
    if len(rects) == 0:
        return 0

    #dlib용 사각형으로 변환 및 크기 조정.. (이것도 다시 조정 필요..)
    newRect = dlib.rectangle(int(rects[0].left()),
                            int(rects[0].top()),
                            int(rects[0].right()),
                            int(rects[0].bottom()))

    #랜드마크 포인트 반환..
    points = []
    [points.append((p.x, p.y)) for p in predictor(im, newRect).parts()]
    return points

#EAR값 구하기

- 눈의 크기를 점수로 나타냄..

In [ ]:
#EAR값 구하는 함수..
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)

    return ear

#checkEyeStatus

- 눈을 떳는지 안떳는지 확인..

leftEyeIndex = [36, 37, 38, 39, 40, 41]

rightEyeIndex = [42, 43, 44, 45, 46, 47]

In [ ]:
def checkEyeStatus(landmarks):
    mask = np.zeros(frame.shape[:2], dtype = np.float32)
    
    #양쪽 눈의 좌표를 가져와서 모양을 따온다..
    #fillConvexPoly : 주어진 점으로 이루어진 볼록다각형 만들어줌..
    hullLeftEye = []
    for i in range(0, len(leftEyeIndex)):
        hullLeftEye.append((landmarks[leftEyeIndex[i]][0], landmarks[leftEyeIndex[i]][1]))
    cv2.fillConvexPoly(mask, np.int32(hullLeftEye), 255)

    hullRightEye = []
    for i in range(0, len(rightEyeIndex)):
        hullRightEye.append((landmarks[rightEyeIndex[i]][0], landmarks[rightEyeIndex[i]][1]))
    cv2.fillConvexPoly(mask, np.int32(hullRightEye), 255)


    #EAR값 구하기..
    leftEAR = eye_aspect_ratio(hullLeftEye)
    rightEAR = eye_aspect_ratio(hullRightEye)

    ear = (leftEAR + rightEAR) / 2.0
    
    
    eyeStatus = 1          # 1 -> Open, 0 -> closed
    #thresh = 0.3
    if (ear < thresh):
        eyeStatus = 0

    return eyeStatus

#checkDrowsy()

- 조는 시간을 확인..

- 메인함수에 시간을 재고 있는 리스트가 필요할 것 같습니다..


- 눈 상태가 감겨있으면
    - 시간을 계속 잰다
- 그렇지 않으면
    - 시간을 초기화 한다..

In [ ]:
import time

#누적되는 시간을 저장할 리스트
#메인함수에서 지정..
t = []

#현재 눈 상태와 시간을 저장한 리스트를 받아옴
def checkDrowsy(eyeStatus, t):
    drowsy = 0

    if eyeStatus:       #눈 뜬상태
        t = []      #시간 초기화

    else:   #눈을 감았을 때만..
        t.append(time.time())     #현재시간을 추가..

        #drowsyLimit : 임의로 정해야하는 졸음 판별 임계값
        #차가 달리는 속도에 따라서 임계값을 바꿔야할지는 고민이 필요할 것 같아요..
        if t[-1] - t[0] > drowsyLimit:
            drowsy = 1
    
    return drowsy   #졸음 : 1,  아님 : 0